In [1]:
import units as u
import numpy as np

## 3.4 opportunity cost emissions

In [2]:
class Plant:
    def __init__(self, t_po, l, t_r):
        self.t_po = t_po * u.year # planning_to_op
        self.l = l * u.year # plant operating lifetime
        self.t_r = t_r * u.year # time to refurbish
    # def __str__(self):
    #     return f"{self.lhv}, {self.eff}, {self.density}"

In [3]:
def calc_background_emissions(plant, _e_g=550, _y=100):
    e_g = _e_g * u.g / u.kwh # background grid emissions -(grams of CO2)
    y = _y * u.year # years for comparsion 
    numerator = plant.t_po + (y - plant.t_po) * (plant.t_r/(plant.l + plant.t_r))
    e_br = e_g * (numerator)/y
    return e_br 


In [9]:
a = Plant(t_po=2, l=30, t_r=0.5)
b = Plant(t_po=17, l=40, t_r=3)
e_br_a = calc_background_emissions(a, _e_g=550, _y=100)
e_br_b = calc_background_emissions(b, _e_g=550, _y=100)
print("A: ", e_br_a, "\n B:", e_br_b)
e_oc = e_br_b - e_br_a
print(f"Opportunity Cost: {e_oc}")

A:  19.836 gram / hour / kilowatt 
 B: 125.349 gram / hour / kilowatt
Opportunity Cost: 105.513 gram / hour / kilowatt


## 3.5 anthropogenic heat flux

In [6]:
chi_co2 = 113 # mixing ratio -> ppmv (parts per million by volume)
C = 8.0055e15 * u.g # grams of CO2/ppmv-CO2 
tau_co2 = 50 * u.year
E_co2 = chi_co2 * C / tau_co2
print(f"E_co2 :{E_co2}")

A_h = 0.0015 * u.watt / u.m**2
F_co2 = 1.82 * u.watt / u.m**2
G_elec = 9e12 * u.kwh / u.year

H = (E_co2 * A_h)/(F_co2 * G_elec) # grams-CO2/kwh 

print(f"H :{H}")

E_co2 :18092430000000000.000 gram / year
H :1.657 gram / hour / kilowatt


## 3.12 carbon capture

In [32]:
class Chemical:
    def __init__(self, mw, gwp20, gwp100):
        self.MW = mw/1000 *  u.ureg('g/mol') # planning_to_op
        self.GWP20 = gwp20 * u.g / u.g # kg-C02/kg-(chemical)
        self.GWP100= gwp100 * u.g / u.g # kg-C02/kg-(chemical)

In [33]:
CH4 = Chemical(16.04276, 86, 34)
CO2 = Chemical(44.0098, 1, 1)
CO2.MW

0.0440098 <Unit('gram / mole')>

In [44]:
# coal plant emissions 
ER_co2_coal_stack = 950 * u.g / u.mwh # emission rate from combustion of coal at stack (g-co2/mwh) 
EC_co2 = 750 * u.g / u.mwh # emissions captured with carbon capture equipment(g-co2/mwh) 

In [37]:
# emissions rate at natural gas plant 
ER_co2_ng = 300 * u.g / u.mwh # emission rate from combustion of natural gas in natural gas plant ~ stack emissions  (g-co2/mwh) 
UER_co2_ng = 30 * u.g / u.mwh # upstream emissions due to mining and transport of ng (g-co2/mwh) 

In [46]:
# determine how much emissions result from leaked methane in upstream processes 

mb_co2_ng = ER_co2_ng / CO2.MW # moles of co2 due to upstream emissions (mol-C02/mwh)
print(mb_co2_ng)


FM_ch4 = 93.9/100 # fraction of moles of ng burned that are ch4  
i_ch4_ng = mb_co2_ng * FM_ch4 * CH4.MW # mass intensity* of methane burned in upstream process (kg-ch4/mwh)
print(i_ch4_ng) 


LR_ng = 2.3/100  # upstream leakage rate of ng
ul_ch4 = (i_ch4_ng * LR_ng) / (1 - LR_ng) # upstream leakage amount of ng, and is emitted as ch4 (kg-ch4/mwh)
print(ul_ch4)

# 20 and 100 year co2 emissions due to upstream leakage of methane 
ul_co2e_ch4_20 = ul_ch4 * CH4.GWP20 
ul_co2e_ch4_100 = ul_ch4 * CH4.GWP100 # kg-co2e/mwh 

print(ul_co2e_ch4_20, ul_co2e_ch4_100)


6816.664 mole / hour / megawatt
102.687 gram / hour / megawatt
2.417 gram / hour / megawatt
207.897 gram / hour / megawatt 82.192 gram / hour / megawatt


In [42]:
# remaining upstream emissions are emitted as co2 directly 
unl_co2 = UER_co2_ng # * (1 - LR_ng) #TODO check this!!
print(unl_co2)

29.310 gram / hour / megawatt


In [43]:
# total emissions from ng plant = upstream leakage due to ch4 + upstream emissions due to co2 + plant emissions as co2 
co2e_ng_20 = ul_co2e_ch4_20  + ER_co2_ng + unl_co2
co2e_ng_100 = ul_co2e_ch4_100 + ER_co2_ng  + unl_co2 
co2e_ng_20, co2e_ng_100

(350.0997018990834 <Unit('gram / hour / megawatt')>,
 337.5291844717307 <Unit('gram / hour / megawatt')>)

In [45]:
# percent emissions effectively captured = captured emissions/ total emissions 
p_EC_20 = EC_co2 / (ER_co2_coal_stack + co2e_ng_20) 
p_EC_100 = EC_co2 / (ER_co2_coal_stack + co2e_ng_100) 

p_EC_20, p_EC_100

(0.5768788339113216 <Unit('dimensionless')>,
 0.5825110677454062 <Unit('dimensionless')>)